In [386]:
import os
import cv2
import numpy as np

#os.chdir(r'C:\Users\Vovan\\Downloads\floor')


sqrt = np.sqrt
sin  = np.sin
cos  = np.cos
tan  = np.tan
pi   = np.pi
arr  = np.array
asin = np.arcsin
acos = np.arccos
atan = np.arctan
norm = np.linalg.norm


def cast(a):    
    '''
    Gets output of cv2.findChessboardCorners
    Returns an array of corners of necessary shape
    '''
    b = np.zeros([len(a), 2])
    for i in range(len(b)):
        b[i] = a[i][0]
    return b


def corners_sort(cast_cors, shape=(7,7)): 
    ''' 
    Gets casted output of cv2.findChessboardCorners 
    Returns sorted edge corners of the desk 
    2 _ 
    1 3 
    ''' 
    board_size = shape[0] 

    cors = np.zeros((4, 2)) 

    cors[0] = cast_cors[ 0]                #geting edge corners 
    cors[1] = cast_cors[ board_size-1] 
    cors[2] = cast_cors[-board_size] 
    cors[3] = cast_cors[-1] 

    sort12 = arr(sorted(sorted(cors, key = lambda x: x[0])[:2], key = lambda x: x[1])) 
    sort34 = arr(sorted(sorted(cors, key = lambda x: x[0])[2:], key = lambda x: x[1])) 

    sort = np.zeros((3, 2)) 

    sort[0] = sort12[1] #sorting by pattern 
    sort[1] = sort12[0] 
    sort[2] = sort34[1]  

    return sort

def pix_trans(pixels, image_shape):
    '''
    Gets coordinates in the image in left top corner reference system
    Returns coordinates of projection of light rays on photomatrix in central reference system
    '''
    shape = list(image_shape)
    window_size = np.zeros(3)
    window_size[:2] = shape[1::-1]

    const = 1/2 * (window_size * arr([-1, 1, 0])) * np.ones([3, 3])

    coef  = arr([1, -1, 0]) * np.ones([3, 3]) 
    
    #invert (*-1) because lens invert image
    points = - (coef * pixels + const)
    
    return points 

'''
def pixels2rays(points, f):
    
    Gets array of shape (n, 3) of points like [x, y, 0]
    Cast points on the photomatrix in central reference system to components of light ray vector 
    knowing optical vector f
    Return array of light of shape (n, 3)
    
    x0 = arr([-f[1], f[0], 0])    #x axis on the image in central reference system orthogonal to f
    x0 = x0/norm(x0)
    
    y0 = arr([-f[0]*f[2], -f[1]*f[2], f[0]**2+f[1]**2])
    y0 = y0/norm(y0)                 #y axis on the image in central reference system orthogonal  to f
    
    
    C = np.zeros([3,3])   #transformation matrix
    C[0] = x0
    C[1] = y0
    C = C.T
    
    rays = np.zeros(points.shape)
    for i in range(len(rays)):
        rays[i] = C.dot(points[i]) + f
        
    return rays
'''
def pixels2rays(points, f):
    '''
    Gets array of shape (n, 3) of points like [x, y, 0]
    Cast points on the photomatrix in central reference system to components of light ray vector 
    knowing optical vector f
    Return array of light of shape (n, 3)
    '''
    x0 = arr([-f[1], f[0], 0])    #x axis on the image in central reference system orthogonal to f
    x0 = x0/norm(x0)

    y0 = arr([-f[0]*f[2], -f[1]*f[2], f[0]**2+f[1]**2])
    y0 = y0/norm(y0)                 #y axis on the image in central reference system orthogonal  to f


    C = np.zeros([3,3])   #transformation matrix
    C[0] = x0
    C[1] = y0
    
    rays = points @ C + f
        
    return rays

def real_coordinates(pixels, foc, Len):
    '''
    Gets array of points in central reference system, optical vector, position of the lens
    Returns coordinates of the points on the floor in robot's reference system
    '''
    
    ray = pixels2rays(pixels, foc)      #cast pixels to light rays
    
    real_points = np.zeros(ray.shape)   #find coordinates of points on the floor
    for i in range(len(real_points)):
        real_points[i] = Len - Len[2]/ray[i][2] * ray[i]
        
    return real_points

def calibration(size, am): 
    '''
    Gets size of the desk and point on photomatrix
    Returns 3d coordinates of the lens in the space - L, and componets of optical axis of the lens - f
    norm of f approximately shows distance between lens and photomatrix
    '''
    
    x1 = am[0][0]   #necessary variables
    x2 = am[1][0]
    x3 = am[2][0]

    y1 = am[0][1]
    y2 = am[1][1]
    y3 = am[2][1]

    X1 = x1 - x2
    X2 = x2 - x3
    X3 = x3 - x1

    Y1 = y1 - y2
    Y2 = y2 - y3
    Y3 = y3 - y1

    Z1 = y1*x2 - y2*x1
    Z2 = y2*x3 - y3*x2
    Z3 = y3*x1 - y1*x3
    
    dx1 = X1/Z1 - X2/Z2
    dx2 = X2/Z2 - X3/Z3
    dx3 = X3/Z3 - X1/Z1

    dy1 = Y1/Z1 - Y2/Z2
    dy2 = Y2/Z2 - Y3/Z3
    dy3 = Y3/Z3 - Y1/Z1

    
    k3 = - Y1/Z1 * dx2                 #coefficients of third degree polynomial ki where i - power of argument
    k2 =   Y1/Z1 * dx1 + dy1 * dx3
    k1 = - Y3/Z3 * dx2 + dy2 * dx3 
    k0 =   Y3/Z3 * dx1

   
    cb = np.roots([k3, k2, k1, k0])             #find polynomial zeros. cb - tan of turn angle
    
    nans = []
    
    for i in range(len(cb)):                    #delete imaginary roots 
        if ((np.imag(cb[i]) != 0)):
            nans += [i]
            
    cb = np.real(np.delete(cb, nans))
    
    
    cc = (cb * dy1 + dy2) / (cb * dx2 - dx1)     #find cc - sin of incline angle
    
    
    nans = []                                    #delete solution > 1 by abs value. sin can't be > 1
    for i in range(len(cc)):                     #delete solution > 0. angle should be < 0. camera pointing down
        if ((abs(cc[i]) > 1)  or (cc[i] > 0)):
            nans += [i]
            
    cc = np.delete(cc, nans)      #delete cc unsuitable roots and cb both
    cb = np.delete(cb, nans)    
    
    
    
    ca = np.zeros(len(cb))       #find ca - norm of optical vector f
    for i in range(len(ca)):
        ca[i] = - (sqrt(1 -(cc[i])**2) * Z2 / (cc[i] * X2 + (cb[i]-1) / (cb[i]+1) * Y2))
    
   
    nans = []                    #delete solutions < 0. norm can't be < 0
    for i in range(len(ca)):
        if (ca[i] < 0):
            nans += [i]
    
    ca = np.delete(ca, nans)     #delete unsiutable solutions for all parameters
    cb = np.delete(cb, nans)
    cc = np.delete(cc, nans)

    prams = np.transpose(arr([ca, atan(cb), asin(cc)]))    #collect all parameters to an array
    p = np.real(prams)                                     #delete imaginary part(=0) to not spoil output
    
    
    f = np.zeros(p.shape)      #find components of optical vector for all posible solutions
    
    for i in range(len(p)):
        f[i] = -p[i][0]*arr([-cos(p[i][2])*sin(p[i][1]), cos(p[i][1])*cos(p[i][2]), sin(p[i][2])])
    
    
    q = np.zeros([len(f), 3, 3])         #find components of the vector of light ray
    for i in range(len(f)):
        q[i] = pixels2rays(am, f[i])     #cast coordinates of points on the photomatrix to components of ray vectors

        
    nans = []                            #to not divide by 0
    for i in range(len(q)):
        if ((q[i][0][2] == 0) or (q[i][2][2] == 0)):
            nans += [i]
    
    f = arr([f[i] for i in range(len(f)) if i not in nans])
    q = arr([q[i] for i in range(len(q)) if i not in nans])
    
        
    L = np.zeros(f.shape)        #find coordinates of the lens
    for i in range(len(L)):
        L[i][2] = size / (q[i][0][0]/q[i][0][2] - q[i][2][0]/q[i][2][2])
        L[i][1] = L[i][2] * q[i][0][1] / q[i][0][2]
        L[i][0] = L[i][2] * q[i][0][0] / q[i][0][2] - size / 2
    
        
    nans = []                                    #lens should have negative Y
    for i in range(len(L)):
        if ((np.isnan(L[i]).any()) or (L[i][1] > 0)):
            nans += [i]
    
    
    
    f = arr([f[i] for i in range(len(f)) if i not in nans])    #delete unsiutable solutions for all parameters
    L = arr([L[i] for i in range(len(L)) if i not in nans])
    
    return f, L

def calibration_pix(size, cors, image_shape):
    pixels = np.zeros((3, 3))
    pixels[:, :2] = cors
    return calibration(size, pix_trans(pixels, image_shape))


def calibration_data(img, shape, size):
    '''
    Gets image, shape of desk to find, size of the desk
    Returns huge ugly heap of calibration information
    '''
    _,start_cors = cv2.findChessboardCorners(img, shape)  #find corners of all squares on the image
    
    if (_ == 0):
        print ("no chessboard")
        return None
    
    cors = np.zeros([3, 3])
    cors[:, :2] = corners_sort(cast(start_cors))               #extraction of desk corners. casting from (1,2) to (1,3) shape
    
    points = pix_trans(cors, img.shape)                  #cast image coordinates to normal reference system 
    
    f, L = calibration(size, points)                      #optical vector and lens position
    
    #data = ()                                             #collecting all data
    
    if (len (f) == 0):
        print ("no solutions")
        return None
    
    #for i in range(len(f)):
    
    data = []
    
    for ind in range(len(f)):
        solution = {}
        
        f_norm  = {'opt_len' : norm(f[ind])}
        incline = {'incline' : np.around(180/pi*acos(np.sqrt(f[ind][0]**2 + f[ind][1]**2)/norm(f[ind])), 1)}
        turn    = {'turn   ' : np.around(180/pi*atan(f[ind][0] / f[ind][1]), 2)}
        lens    = {'lens   ' : L[ind]}
        foc     = {'optical' : f[ind]}
        cors_   = {'corners' : cors}                            #adding array of corners


        solution.update (f_norm)
        solution.update (incline)
        solution.update (turn)
        solution.update (lens)
        solution.update (foc)
        solution.update (cors_)
        
        data += [solution]
    

    return data

In [370]:
'''
def pixels2rays(points, f):
    
    Gets array of shape (n, 3) of points like [x, y, 0]
    Cast points on the photomatrix in central reference system to components of light ray vector 
    knowing optical vector f
    Return array of light of shape (n, 3)
    
    x0 = arr([-f[1], f[0], 0])    #x axis on the image in central reference system orthogonal to f
    x0 = x0/norm(x0)
    
    y0 = arr([-f[0]*f[2], -f[1]*f[2], f[0]**2+f[1]**2])
    y0 = y0/norm(y0)                 #y axis on the image in central reference system orthogonal  to f
    
    
    C = np.zeros([3,3])   #transformation matrix
    C[0] = x0
    C[1] = y0
    
    rays = am @ C + f
        
    return rays
'''

[[ 0.88160008 -3.32313758  2.01077315]]
[[[ 2.29347197 -2.40537534  2.90850932]
  [ 1.11590813 -3.62137304  1.41515973]
  [-3.81082233 -3.34056366  4.03931161]]]
[ 0.51929895 -0.54463665  0.65855867]
[ 0.27587138 -0.89526473  0.34985145]
[-0.58803826 -0.51547384  0.62329585]

[ 0.51929895 -0.54463665  0.65855866]
[ 0.27587138 -0.89526473  0.34985145]
[-0.58803826 -0.51547385  0.62329585]
[[-0.22358178 -2.3874895   2.88688229]]
